In [2]:
import random
import math
import numpy as np
from numpy import *
from math import radians, sin, cos, sqrt, atan2
import functools 
import xlrd
import pandas as pd
from statistics import mean
import matplotlib.pyplot as plt 
import pmagpy.pmag as pmag
import pmagpy.pmagplotlib as pmagplotlib
import pmagpy.ipmag as ipmag
import matplotlib 
from typing import List, Tuple, Dict
from scipy.interpolate import interp1d

if __name__ == "__main__":
    
    file_path=r'C:\Users\Administrator\Desktop\GRL\daima\allpole.xlsx'
    
    standarderror_list=[]   
    distance5_list=[]
    number_list=[]
    m=0        
    while m<50: #Degrees of northward movement of the Tethyan Himalaya      
        
        data2=pd.read_excel(file_path,sheet_name="Combinedpole",usecols='A:E') #reading combined paleomagnetic pole used
        data3=pd.read_excel(file_path,sheet_name="Referencepoint1",usecols='A:B') #reading reference point
        Prot=[0,2.4,m] #m,the angle at which the paleomagnetic pole rotates around the Eular pole
        lats=data2['Thlat'].values #Extract the latitude of the paleomagentic pole to be rotated 
        lons=data2['Thlon'].values #Extract the longitude of the paleomagentic pole to be rotated
        RLats,RLons=pmag.pt_rot(Prot,lats,lons)              
        lats1=data3['CKLat'].values 
        lons1=data3['CKLon'].values 
        CKLats,CKLons=pmag.pt_rot(Prot,lats1,lons1)
        
        df0=pd.DataFrame()
        df1=pd.DataFrame()
        df2=pd.DataFrame()
        df3=pd.DataFrame()
        df4=pd.DataFrame()
        df5=pd.DataFrame()
        df6=pd.DataFrame()
        df7=pd.DataFrame()
        df8=pd.DataFrame()
       
        i=0
        while i <9:                                                    
            radius=float(data2.iloc[i,3]); Kpa1=int(data2.iloc[i,2]); centery=float(RLats[i]); centerx=float(RLons[i]);Age_TH=float(data2.iloc[i,4]) 
            #Extraction of the corresponding parameters of the palaeomagnetic poles  
            rotdi=ipmag.fishrot(k=Kpa1,n=1000,dec=centerx,inc=centery) 
            #Extraction of 30 palaeomagnetic poles from the palaeomagnetic pole error range consistent with the Fischer distribution，dec/inc represents lon/lat
            bignew_1_list=[] #Place the extracted paleomagnetic poles in this list
            for di in rotdi: 
                new_1_list=[] 
                new_1_list.append(di[0]) 
                new_1_list.append(di[1]) 
                bignew_1_list.append(new_1_list) 
        ###########################################################################################################################################################################################
            Indian_pole = pd.read_excel(file_path, sheet_name="Indian_APWP",  usecols='A:E')   #reading Indian 10 Ma windows APWP
            Age=Indian_pole.loc[:,'Age']; Lat=Indian_pole.loc[:,'Lat']; Lon=Indian_pole.loc[:,'Lon']; A95=Indian_pole.loc[:,'A95']; K=Indian_pole.loc[:,'K']
            #Take out the parameters corresponding to the Indian poles
            #Define second-order one-dimensional interpolation 
            erjiechazhi1 = interp1d(Age, Lat, kind='quadratic', axis=-1)
            erjiechazhi2 = interp1d(Age, Lon, kind='quadratic', axis=-1)
            erjiechazhi3 = interp1d(Age, A95, kind='quadratic', axis=-1)
            erjiechazhi4 = interp1d(Age, K, kind='quadratic', axis=-1)
            Age_new=np.arange(0,200.1,0.1);Lat_new=erjiechazhi1(Age_new);Lon_new=erjiechazhi2(Age_new);A95_new=erjiechazhi3(Age_new);K_new=erjiechazhi4(Age_new) #Apply the defined interpolation function
            Age_new=list(Age_new);Lat_new=list(Lat_new);Lon_new=list(Lon_new);A95_new=list(A95_new); K_new=list(K_new);#Converting arrays to lists 
            Age_new=[round(age, 1) for age in Age_new] #Retain 0.1 window age as a decimal 

            newIndian_pole=list(zip(Age_new,Lat_new,Lon_new, A95_new,K_new)) #Indian APWP with 0.1 Ma window 
            n=Age_new.index(Age_TH) #Finding the position of the Indian paleomagnetic pole corresponding to the age of Tethyan Himalaya 
            corr_Indianpole=newIndian_pole[n]

            radius=corr_Indianpole[3]; Kpa2=corr_Indianpole[4]; centery=corr_Indianpole[1]; centerx=corr_Indianpole[2]

            rotdi=ipmag.fishrot(k=Kpa2,n=1000,dec=centerx,inc=centery) 
            bignew_2_list=[]
            for di in rotdi:
                new_2_list=[] 
                new_2_list.append(di[0]) 
                new_2_list.append(di[1]) 
                bignew_2_list.append(new_2_list)  
        ###############################################################################################################################################################################################            
            length=len(bignew_1_list)
            j=0
            distance2_list=[]
            while j<length:
                
                lon1=bignew_1_list[j][0]; lat1=bignew_1_list[j][1] 
                Th_d,Th_i=pmag.vgp_di(lat1,lon1,CKLats[i],CKLons[i])             
                TH_Palo=pmag.plat(Th_i)                 

                lon2=bignew_2_list[j][0]; lat2=bignew_2_list[j][1]
                In_d,In_i=pmag.vgp_di(lat2,lon2,CKLats[i],CKLons[i])            
                In_Palo=pmag.plat(In_i)                

                distance1=TH_Palo-In_Palo
                distance2_list.append(distance1) #The palaeolatitude difference calculated by taking 1000 values out of each palaeomagnetic pole is stored
                j+=1
            
            if i==0:
                df0=pd.DataFrame(distance2_list, columns=['CUGB0']) 
            elif i==1:
                df1=pd.DataFrame(distance2_list, columns=['CUGB1']) 
            elif i==2:                  
                df2=pd.DataFrame(distance2_list, columns=['CUGB2']) 
            elif i==3:
                df3=pd.DataFrame(distance2_list, columns=['CUGB3']) 
            elif i==4:
                df4=pd.DataFrame(distance2_list, columns=['CUGB4']) 
            elif i==5:
                df5=pd.DataFrame(distance2_list, columns=['CUGB5']) 
            elif i==6:
                df6=pd.DataFrame(distance2_list, columns=['CUGB6']) 
            elif i==7:
                df7=pd.DataFrame(distance2_list, columns=['CUGB7']) 
            elif i==8:
                df8=pd.DataFrame(distance2_list, columns=['CUGB8']) 
            
            i+=1
        
        CUGB=pd.concat([df0,df1,df2,df3,df4,df5,df6,df7,df8],ignore_index=True,axis=1) #1000 rows and 10 columns
       
        avg=np.mean(CUGB,axis=1) #Mean between 10 palaeolatitude differences
        
        
        final_list=[]
        for average in avg:
            final_list.append(average)
        
        standarderror=np.std(final_list,ddof=0) #Standard deviation between 1000 mean
        distance5=abs(np.mean(final_list))
        distance5_list.append(distance5)
        standarderror_list.append(standarderror)    
        
        number_list.append(m)
        m+=0.1
    
    
    x=number_list
    y=distance5_list
    y1=[g*1.96 for g in standarderror_list]
    y2=[abs(y[h])+abs(y1[h]) for h in range (0,len(y))]
    y3=[abs(y[h])-abs(y1[h]) for h in range (0,len(y))]
    
    plt.figure(figsize=(5,3),dpi=300)
    plt.plot(number_list,distance5_list,color='red') 
    plt.fill_between(number_list,y2,y3,alpha=0.2,color='blue') # plotting the range of confidence intervals, alpha represents transparency
    
    min_y= min(y)
    answer=distance5_list.index(min_y)
    
    print("degrees of movement of the TH,paleolatitudinal gap,%95 confidence interval")
    print(0,y[0],y1[0],sep=",")
    print(x[answer],min_y,y1[answer],sep=",")
    print(30,y[29],y1[29],sep=",")
    
    plt.xlabel('Degrees of movement of the TH')
    plt.ylabel('Paleolatitudinal gap')    
    plt.savefig('NS-combined9pole(28.1-92.4).svg',format='svg') 
    plt.show() 

degrees of movement of the TH,paleolatitudinal gap,%95 confidence interval
0,10.832699952051293,7.941108356433826
12.099999999999973,0.025374983246588715,7.876571358772883
30,8.308235799481015,7.696387496994788
